# IBM Data Science Specialization Capstone Project

## Insert Title Here

In [1]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed folium
import folium
from folium import plugins
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### Foursquare Credentials

In [2]:
# The code was removed by Watson Studio for sharing.

### Importing Postal Code data from Canada using pandas read_html mehod

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url)[0]

df.head()

,Postal Code,District,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Removing Postal Codes without Borough and Duplicates

In [4]:
df = df[df['District'] != 'Not assigned']
df = df.rename(columns={'Neighbourhood':'Neighborhood'}) #Wikipedia seems to have a typo, so I fixed it!

df['Neighborhood'] = df.groupby('Postal Code')['Neighborhood'].transform(lambda x: ','.join(x))
df = df.drop_duplicates().reset_index(drop=True)

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
df['Neighborhood'].replace('Not assigned',df['District'], inplace = True)

### Importing geospatial data for each postal code

In [6]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')

df = df.merge(df_geo) #Merge geospatial data to the postal code dataframe

### Selecting data from Toronto

In [7]:
df = df[df['District'].str.contains('Toronto')].reset_index(drop=True)
df.head()

,Postal Code,District,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Getting geographic coordinates for Toronto, CA

In [8]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.6534817, -79.3839347.


### Defining a function to get venues from Foursquare explore API

In [9]:
radius = 500
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results]),

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue ID',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Getting the venues around each neighborhood in Toronto

In [10]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [11]:
toronto_bookstores = toronto_venues[toronto_venues['Venue Category']=='Bookstore']
toronto_bookstores.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
119,"Garden District, Ryerson",43.657162,-79.378937,Indigo,4b2a6eb8f964a52012a924e3,43.653515,-79.380696,Bookstore
129,"Garden District, Ryerson",43.657162,-79.378937,BMV Books,4afec2eef964a520373022e3,43.657047,-79.381661,Bookstore
222,St. James Town,43.651494,-79.375418,Indigo,4b2a6eb8f964a52012a924e3,43.653515,-79.380696,Bookstore
452,"Richmond, Adelaide, King",43.650571,-79.384568,Indigo,4b2a6eb8f964a52012a924e3,43.653515,-79.380696,Bookstore
490,"Richmond, Adelaide, King",43.650571,-79.384568,Indigospirit,4b059c24f964a520c2df22e3,43.648350,-79.380347,Bookstore


In [12]:
def getVenueRatings(venues):
    
    ratings = []
    likes =[]
    for venueId in venues:

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venueId, CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(url).json()
        try:
            rating = result['response']['venue']['rating']
            likes = result['response']['venue']['likes']['count']
        except:
            rating = None
            likes = None
        
        ratings.append((venueId, rating, likes))
 
    # convert the venues list into a DataFrame
    rating = pd.DataFrame(ratings)
    # define the column names
    rating.columns = ['Venue ID', 'Venue Rating','Venue Likes']

    return rating    

bookstore_ratings = getVenueRatings(toronto_bookstores['Venue ID'].drop_duplicates())
bookstore_ratings.head()

,Venue ID,Venue Rating,Venue Likes
0,4b2a6eb8f964a52012a924e3,8.5,244
1,4afec2eef964a520373022e3,7.3,37
2,4b059c24f964a520c2df22e3,7.4,8
3,4b26a6a9f964a520ff7e24e3,8.4,11
4,58e91011e9233e1591c12031,8.1,9


In [47]:
toronto_bookstores = toronto_bookstores.merge(bookstore_ratings,how='left')
'Number of Neighborhoods = {}, Number of venues = {}, Number of venue categories = {}, Number of bookstores = {}'.format(df.shape[0], toronto_venues.shape[0], toronto_venues['Venue Category'].nunique(), toronto_bookstores.shape[0])
toronto_bookstores

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Venue Likes
0,"Garden District, Ryerson",43.657162,-79.378937,Indigo,4b2a6eb8f964a52012a924e3,43.653515,-79.380696,Bookstore,8.5,244
1,"Garden District, Ryerson",43.657162,-79.378937,BMV Books,4afec2eef964a520373022e3,43.657047,-79.381661,Bookstore,7.3,37
2,St. James Town,43.651494,-79.375418,Indigo,4b2a6eb8f964a52012a924e3,43.653515,-79.380696,Bookstore,8.5,244
3,"Richmond, Adelaide, King",43.650571,-79.384568,Indigo,4b2a6eb8f964a52012a924e3,43.653515,-79.380696,Bookstore,8.5,244
4,"Richmond, Adelaide, King",43.650571,-79.384568,Indigospirit,4b059c24f964a520c2df22e3,43.648350,-79.380347,Bookstore,7.4,8
5,"The Danforth West, Riverdale",43.679557,-79.352188,Re: Reading,4b26a6a9f964a520ff7e24e3,43.678507,-79.347678,Bookstore,8.4,11
6,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Indigospirit,4b059c24f964a520c2df22e3,43.648350,-79.380347,Bookstore,7.4,8
7,Studio District,43.659526,-79.340923,Queen Books,58e91011e9233e1591c12031,43.660651,-79.342267,Bookstore,8.1,9
8,"High Park, The Junction South",43.661608,-79.464763,Pandemonium,4b772204f964a520a2802ee3,43.665533,-79.466931,Bookstore,7.6,8
9,"Parkdale, Roncesvalles",43.648960,-79.456325,A Good Read,4baea612f964a520c2ca3be3,43.649470,-79.450339,Bookstore,7.3,5


### Analysing each Neighborhood

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
del toronto_onehot['Neighborhood']
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Beer Bar,Farmers Market,Cocktail Bar,Cheese Shop,Café,Seafood Restaurant,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Climbing Gym,Burrito Place,Italian Restaurant,Restaurant,Intersection,Stadium,Bar
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Recording Studio,Butcher,Restaurant,Burrito Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport,Bar,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Coffee Shop
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Poke Place,Portuguese Restaurant


### Clustering Neighborhoods

In [18]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, init='k-means++',random_state=10).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='inner')

In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
1,Downtown Toronto,0,Coffee Shop,College Cafeteria,Yoga Studio,Distribution Center,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Fast Food Restaurant,Hotel,Bookstore,Pizza Place
3,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,American Restaurant,Restaurant,Clothing Store,Moroccan Restaurant,Cosmetics Shop,Lingerie Store,Beer Bar
4,East Toronto,0,Pub,Health Food Store,Trail,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
5,Downtown Toronto,0,Coffee Shop,Restaurant,Bakery,Beer Bar,Farmers Market,Cocktail Bar,Cheese Shop,Café,Seafood Restaurant,Pub
6,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Poke Place,Portuguese Restaurant
7,Downtown Toronto,0,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Diner,Restaurant,Candy Store,Baby Store,Nightclub
8,Downtown Toronto,0,Coffee Shop,Café,Gym,Restaurant,Clothing Store,Thai Restaurant,Bar,Hotel,Sushi Restaurant,Breakfast Spot
9,West Toronto,0,Pharmacy,Bakery,Middle Eastern Restaurant,Bus Stop,Café,Bar,Bank,Supermarket,Portuguese Restaurant,Brewery


### Cluster 1

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,1,Summer Camp,Yoga Studio,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster 2

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
33,Downtown Toronto,2,Park,Playground,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster 3

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Bus Line,Swim School,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 4

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Ice Cream Shop,Home Service,Health & Beauty Service,Garden,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio


### Clusters with a Bookstore

In [26]:
bookstore_cluster = toronto_merged[toronto_merged.Neighborhood.isin(toronto_bookstores.Neighborhood)]
bookstore_cluster

,Postal Code,District,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Fast Food Restaurant,Hotel,Bookstore,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,American Restaurant,Restaurant,Clothing Store,Moroccan Restaurant,Cosmetics Shop,Lingerie Store,Beer Bar
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Gym,Restaurant,Clothing Store,Thai Restaurant,Bar,Hotel,Sushi Restaurant,Breakfast Spot
12,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
16,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Gastropub,Seafood Restaurant,Japanese Restaurant,Deli / Bodega
17,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Yoga Studio,Latin American Restaurant,Fish Market,Ice Cream Shop
22,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0,Mexican Restaurant,Grocery Store,Café,Thai Restaurant,Discount Store,Flea Market,Bar,Speakeasy,Bakery,Cajun / Creole Restaurant
25,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Breakfast Spot,Gift Shop,Bookstore,Dog Run,Dessert Shop,Bar,Movie Theater,Italian Restaurant,Cuban Restaurant,Restaurant
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Bakery,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bookstore,Dessert Shop,Pub,Chinese Restaurant
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Café,Coffee Shop,Sushi Restaurant,Pub,Pizza Place,Italian Restaurant,Yoga Studio,French Restaurant,Sandwich Place,Latin American Restaurant


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

for lat, lon, poi, venue in zip(toronto_bookstores['Venue Latitude'], toronto_bookstores['Venue Longitude'], toronto_bookstores['Neighborhood'], toronto_bookstores['Venue']):
    label = folium.Popup(str(poi) + ' Bookstore: ' + str(venue), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=False,
        #fill_color='green',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
map_heat= folium.Map(location=[latitude, longitude], zoom_start=12)
heat_data = [[row['Venue Latitude'],row['Venue Longitude']] for index, row in toronto_bookstores.iterrows()]
#df_neighborhood_noturkrest = df_neighborhood[df_neighborhood['Neighborhood'].isin(noRestList)]

folium.TileLayer('cartodbpositron').add_to(map_heat) 
HeatMap(heat_data).add_to(map_heat)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=False,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_heat)

for lat, lon, poi, venue in zip(toronto_bookstores['Venue Latitude'], toronto_bookstores['Venue Longitude'], toronto_bookstores['Neighborhood'], toronto_bookstores['Venue']):
    label = folium.Popup(str(poi) + ' Bookstore: ' + str(venue), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_heat)


folium.Circle([latitude,longitude], radius=2000, fill=False, color='white').add_to(map_heat)
folium.Circle([latitude,longitude], radius=4000, fill=False, color='white').add_to(map_heat)
folium.Circle([latitude,longitude], radius=6000, fill=False, color='white').add_to(map_heat)
folium.Circle([latitude,longitude], radius=8000, fill=False, color='white').add_to(map_heat)
folium.Circle([latitude,longitude], radius=10000, fill=False, color='white').add_to(map_heat)
  
map_heat
